# Imports

In [ ]:
import pandas as pd
import numpy as np

# Read Data

In [ ]:
patient_characteristics_train_path = 'dataset/train/patient_characteristics.csv.gz'
chartevents_train_path = 'dataset/train/chartevents.csv.gz'
labevents_train_path = 'dataset/train/labevents.csv.gz'

In [ ]:
patient_characteristics_test_path = 'dataset/test/patient_characteristics.csv.gz'
chartevents_test_path = 'dataset/test/chartevents.csv.gz'
labevents_test_path = 'dataset/test/labevents.csv.gz'

In [ ]:
patient_characteristics_train = pd.read_csv(patient_characteristics_train_path)
chartevents_train = pd.read_csv(chartevents_train_path)
labevents_train = pd.read_csv(labevents_train_path)

In [ ]:
patient_characteristics_test = pd.read_csv(patient_characteristics_test_path)
chartevents_test = pd.read_csv(chartevents_test_path)
labevents_test = pd.read_csv(labevents_test_path)

# Patient_Characteristics

In [ ]:
patient_characteristics_train.head()

In [ ]:
patient_characteristics_train['LOS'].hist(bins=100)

## Data visualization & feature extraction for patient characteristics.

Decide which columns you want to keep and perhaps come up with some informative features you can extract from this table?

# Chartevents

In [ ]:
chartevents_train.head()

## Data visualization & feature extraction for chartevents.

This table stores patient values per hour. The final task is to diagnose patients for the entire stay (and not per hour!). Perhaps we can extract some simple aggregations? You can simply aggregate by using pd.groupby() function.

# Labevents

In [ ]:
labevents_train.head()

## Data visualization & feature extraction for chartevents.

This table stores patient values per hour. The final task is to diagnose patients for the entire stay (and not per hour!). Perhaps we can extract some simple aggregations?

# Merging & Model Preparation

Let's merge. Try to use the pd.merge() function.

In [ ]:
df_model_train = ... 

df_model_test = ...

In [ ]:
df_model_train.shape, df_model_test.shape

### Drop irrelevant columns and obtain y values.

In [ ]:
to_drop = [...]


diagnoses_train_path = 
diagnoses_test_path = 
diagnoses_train = pd.read_csv(diagnoses_train_path)
diagnoses_test = pd.read_csv(diagnoses_test_path)

df_model_train = df_model_train.drop(to_drop, axis=1)
df_model_test = df_model_test.drop(to_drop, axis=1)

In [ ]:
diagnoses_train = diagnoses_train.sort_values('ICUSTAY_ID')
df_model_train = df_model_train.sort_values('ICUSTAY_ID')

df_model_test = df_model_test.sort_values('ICUSTAY_ID')

In [ ]:
df_model_train.shape, df_model_test.shape

### One-hot encode categorical columns

In [ ]:
categorical_columns = [...]

In [ ]:
df_model_train = pd.get_dummies(df_model_train, columns=categorical_columns, drop_first=True, dummy_na=True)
df_model_test = pd.get_dummies(df_model_test, columns=categorical_columns, drop_first=True, dummy_na=True)

In [ ]:
df_model_train.shape
df_model_test.shape

It's possbile that the amount of columns in the train table differs from the amount in the test table. Let's keep only their intersection.

In [ ]:
column_intersection = list(set(df_model_train.columns).intersection(set(df_model_test.columns)))

df_model_train = df_model_train[column_intersection]
df_model_test = df_model_test[column_intersection]

In [ ]:
df_model_train.shape
df_model_test.shape

Let's make sure all columns are numeric now.

In [ ]:
def to_numeric_df(df):
    for col in df.columns:
        df[col] = pd.to_numeric(df[col])
    return df

In [ ]:
df_model_train = to_numeric_df(df_model_train)
df_model_test = to_numeric_df(df_model_test)

### Treating NaN

What will we do with NaN values? A simple approach is to use DataFrame.fillna().

In [ ]:
df_model_train = ...
df_model_test = ...

# Modelling

### Scale the columns to 0 mean and variance 1.

In [ ]:
from sklearn.preprocessing import StandardScaler, LabelEncoder

X_train = df_model_train.drop('ICUSTAY_ID', axis=1)
X_test = df_model_test.drop('ICUSTAY_ID', axis=1)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)



### Encode text labels to unique numeric values.

In [ ]:

encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train['DIAGNOSIS'])

In [ ]:
from sklearn.[package] import [classifier]

In [ ]:
model = classifier() # Tune hyperparameters!

In [ ]:
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_train)

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score

In [ ]:
print(f'Accuracy: {accuracy_score(y_train, y_pred)}')

cm = pd.DataFrame(confusion_matrix(y_train, y_pred))
cm.columns = encoder.inverse_transform(cm.columns)
cm.index = encoder.inverse_transform(cm.index)

In [ ]:
cm

# Submission

In [ ]:
y_pred_test = model.predict(X_test)

In [ ]:
diagnoses_test = encoder.inverse_transform(y_pred_test)

In [ ]:
submission_df = pd.concat([df_model_test['ICUSTAY_ID'], pd.Series(diagnoses_test, name='DIAGNOSIS_PRED')], axis=1)

In [ ]:
submission_df.to_csv('submission_name.csv', index=False)